In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

# Initialize the simulator

In [19]:
from soft_info import RepCodeIQSimulator

DEVICE = 'ibm_sherbrooke'
D = 11
T = 11

OTHER_DATE = '2024-03-24'
DOUBLE_MSMT = True

_is_hex = True
if DEVICE == 'ibmq_mumbai':
    _is_hex = False

# Initialize simulator
simulator = RepCodeIQSimulator(provider, D, T, DEVICE, _is_hex, OTHER_DATE, DOUBLE_MSMT)

# Needed decoding objects

In [20]:
# Get the KDEs
import numpy as np
from src import cpp_soft_info as csi
from Scratch import load_calibration_memory

bandwidths = [0.6]
num_points = 20
rel_error = 1

all_memories, gmm_dict, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                    other_date=OTHER_DATE, post_process=True,
                                                    double_msmt=False)
kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)


all_memories_PS, gmm_dict_PS, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=OTHER_DATE, post_process=True,
                                                                            double_msmt=True)
kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:22.974787+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 17:49:53.348518+00:00.


In [21]:
best_bandwidths = []
for k, v in kde_dict.items():
    print(v.bestBandwidth, end=' ')
    best_bandwidths.append(v.bestBandwidth)

print()

best_bandwidths_PS = []
for k, v in kde_dict_PS.items():
    print(v.bestBandwidth, end = ' ')
    best_bandwidths_PS.append(v.bestBandwidth)

print()
print()
print(f"mean normal: {np.mean(best_bandwidths)}")
print(f"mean PS: {np.mean(best_bandwidths_PS)}")
print(f"std normal: {np.std(best_bandwidths)}")
print(f"std PS: {np.std(best_bandwidths_PS)}")

0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 
0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6 0.6

In [22]:
from soft_info import inv_qubit_mapping

inverted_q_map = inv_qubit_mapping(simulator.qubit_mapping)

# Check if the simulator is using post selection
print(f"simulator qubit 20 shape: {simulator.all_memories[20]['mmr_0'].shape}")
print(f"Used memories for PS qubit 20 shape: {all_memories_PS[20]['mmr_0'].shape}")


simulator qubit 20 shape: (9411,)
Used memories for PS qubit 20 shape: (9411,)


# Get IQ Data

In [23]:
XBASIS = False
RESETS = False
LOGICAL = '0'

SHOTS = int(1e4)
INDIVIDUAL = False

IQ_data = simulator.generate_IQ(SHOTS, LOGICAL, XBASIS, RESETS, INDIVIDUAL)

# Decode

In [24]:
# Generate pSoft and countMat
HANDLE_OUTLIERS = True

pSoft, countMat = csi.iqConvertor(IQ_data, inverted_q_map, kde_dict, rel_error, -1, HANDLE_OUTLIERS)
pSoft_PS, countMat_PS = csi.iqConvertor(IQ_data, inverted_q_map, kde_dict_PS, rel_error, -1, HANDLE_OUTLIERS)

In [25]:
# Initialize the stim circuits
from soft_info import RepetitionCodeStimCircuit

if INDIVIDUAL is False:
    circ_soft = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=simulator.noise_list, no_fin_soft=True)
    model_soft = circ_soft.circuits[LOGICAL].detector_error_model()
    circ_hard = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=simulator.noise_list, no_fin_soft=False)
    model_hard = circ_hard.circuits[LOGICAL].detector_error_model()
else:
    circ_soft = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=simulator.noise_list, no_fin_soft=True,
                                          msmt_err_dict=simulator.msmt_err_dict, layout=simulator.layout)
    model_soft = circ_soft.circuits[LOGICAL].detector_error_model()
    circ_hard = RepetitionCodeStimCircuit(D, T, XBASIS, RESETS, noise_list=simulator.noise_list, no_fin_soft=False,
                                          msmt_err_dict=simulator.msmt_err_dict, layout=simulator.layout)
    model_hard = circ_hard.circuits[LOGICAL].detector_error_model()

print(f"noise_list: {simulator.noise_list}")
print("[two, single, t1, t2, readout, p_hard, p_soft]")

noise_list: [0.012198849629634207, 0.0004089493467995245, 0.0016527862595933853, 0.0020821405014924475, 0.02578180156360313, 0.019907289814579635, 0]
[two, single, t1, t2, readout, p_hard, p_soft]


In [26]:
# Decode
res_s_K = csi.decodeConvertorAll(model_soft, countMat, pSoft, T, int(LOGICAL), RESETS, decode_hard = False)
res_h_K = csi.decodeConvertorAll(model_hard, countMat, pSoft, T, int(LOGICAL), RESETS, decode_hard = True)

res_s_K_PS = csi.decodeConvertorAll(model_soft, countMat_PS, pSoft_PS, T, int(LOGICAL), RESETS, decode_hard = False)
res_h_K_PS = csi.decodeConvertorAll(model_hard, countMat_PS, pSoft_PS, T, int(LOGICAL), RESETS, decode_hard = True)

In [27]:
print(f"D: {D}, T: {T}, SHOTS: {SHOTS}, LOGICAL: {LOGICAL}, XBASIS: {XBASIS}, RESETS: {RESETS}, INDIVIDUAL: {INDIVIDUAL}, DOUBLE_MSMTS: {DOUBLE_MSMT}\n")

print(f"soft KDE: {res_s_K.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}")
print(f"hard KDE: {res_h_K.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}")

print()

print(f"soft KDE PS: {res_s_K_PS.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")
print(f"hard KDE PS: {res_h_K_PS.num_errors} out of {len(IQ_data)} shots, handleOutliers: {HANDLE_OUTLIERS}, bandwidth: {bandwidths}")

D: 11, T: 11, SHOTS: 10000, LOGICAL: 0, XBASIS: False, RESETS: False, INDIVIDUAL: False, DOUBLE_MSMTS: True

soft KDE: 243 out of 10000 shots, handleOutliers: True
hard KDE: 555 out of 10000 shots, handleOutliers: True

soft KDE PS: 501 out of 10000 shots, handleOutliers: True, bandwidth: [0.6]
hard KDE PS: 557 out of 10000 shots, handleOutliers: True, bandwidth: [0.6]


# Get counts

In [12]:
counts = simulator.get_counts(SHOTS, simulator.stim_circ, RESETS)

In [13]:
import pymatching
from soft_info import counts_to_det_syndr

matching = pymatching.Matching.from_detector_error_model(model_hard)

num_errors = 0
for count_key, shots in counts.items():

    array_processed_str = counts_to_det_syndr(count_key, _resets=RESETS)
    predicted_observable = matching.decode(array_processed_str)
    actual_observable = [(int(count_key[0])+int(LOGICAL))%2]

    if actual_observable != predicted_observable:
        num_errors += shots

print(f"Hard matching COUNTS: {num_errors} out of {SHOTS} shots")

Hard matching COUNTS: 2 out of 10000 shots
